# Assignment 3

As before, if a question can be answered with 'yes/no', or a numeric value, you may simply state as much. If you incorporate code from the internet (which is not required and generally not advisable), please cite the source within your code (providing a URL is sufficient).

We will go through comparable code and concepts in the live learning session. If you run into trouble, start by using the help `help()` function in Python, to get information about the datasets and function in question. The internet is also a great resource when coding (though note that no outside searches are required by the assignment!). If you do incorporate code from the internet, please cite the source within your code (providing a URL is sufficient).

Please bring questions that you cannot work out on your own to office hours, work periods or share with your peers on Slack. We will work with you through the issue.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

### Question 1: Resampling via Bootstrapping

Now, we'll use the `iris` dataset, which we will add to Python using the `statsmodels` library. As always, start by reviewing a description of the dataset, by printing the dataset.

In [ ]:
# Import
iris = sm.datasets.get_rdataset('iris', 'datasets')
df = pd.DataFrame(iris.data)

_(i)_ Create an `alpha_func(D, idx)` function which takes the `Sepal`'s `width` and `length` to calculate for alpha

In [ ]:
# Your code here

# Define the alpha_func to calculate the ratio of Sepal width to Sepal length
def alpha_func(D, idx):
    """
    Calculate alpha as the ratio of Sepal width to Sepal length.

    Parameters:
    D (DataFrame): The DataFrame containing the iris dataset.
    idx (array-like): Indexes to be used for resampling.
    """
    
    sepal_length = D.loc[idx, 'Sepal.Length'].values
    sepal_width = D.loc[idx, 'Sepal.Width'].values
    
    alpha = sepal_width / sepal_length
    return alpha.mean()

# Testing the function with the whole dataset
print(alpha_func(df, df.index))

Test the code below

In [ ]:
alpha_func(df, range(100))

_(ii)_ Construct a new bootstrap data set and recompute alpha

In [ ]:
rng = np.random.default_rng(0)
alpha_func(df,
           rng.choice(100,
                      100,
                      replace=True))

Imagine we are analysts working for a shipping company. The company wants to know the average length of iris' petals, to inform space allotment on an upcoming shipment. The relevant variable in the dataset is `Sepal.Length`. 

_(iii)_ Why is it (perhaps) not sufficient to simply calculate the mean of `Sepal.Length`? What more information will preforming a bootstrap provide to us?  

Calculating the mean of Sepal.Length provides a point estimate of the average petal length. However, this point estimate does not give us any information about the variability or uncertainty around this estimate. Reasons why simply calculating the menan may not be sufficient may include Variability, robustness, confidence intervals.

_(iv)_ We can perform bootstrapping in Python by defining a simple function using `boot_SE()` for computing the bootstrap standard error. Remember, because bootstrapping involves randomness, we must first set a seed for reproducibility!

In [ ]:
# Add your code here to set the seed

# Function to calculate the mean of Sepal.Length
def mean_sepal_length(D, idx):
    return D.loc[idx, 'Sepal.Length'].mean()

# Function to perform bootstrapping and compute the standard error
def boot_SE(D, num_bootstrap=1000, seed=0):
    rng = np.random.default_rng(seed)
    bootstrap_means = []
    
    for _ in range(num_bootstrap):
        sample_idx = rng.choice(D.index, len(D), replace=True)
        bootstrap_means.append(mean_sepal_length(D, sample_idx))
    
    bootstrap_means = np.array(bootstrap_means)
    standard_error = bootstrap_means.std()
    return standard_error, bootstrap_means

# Perform bootstrapping
standard_error, bootstrap_means = boot_SE(df)

print(f"Bootstrap standard error: {standard_error}")




_(v)_ Evaluate the accuracy of our alpha estimate with B = 1000

In [ ]:
# Your code here

# Defining the alpha function
def alpha_func(data, idx):
    X = data.loc[idx, 'Sepal.Length']
    Y = data.loc[idx, 'Sepal.Width']
    return (np.var(Y) - np.var(X)) / (np.var(Y) + np.var(X))

# Defining the bootstrapping function for alpha
def bootstrap_alpha(data, func, num_bootstrap=1000, seed=0):
    rng = np.random.default_rng(seed)
    bootstrap_alphas = []
    
    for _ in range(num_bootstrap):
        sample_idx = rng.choice(data.index, len(data), replace=True)
        bootstrap_alphas.append(func(data, sample_idx))
    
    bootstrap_alphas = np.array(bootstrap_alphas)
    standard_error = bootstrap_alphas.std()
    ci_lower = np.percentile(bootstrap_alphas, 2.5)
    ci_upper = np.percentile(bootstrap_alphas, 97.5)
    
    return bootstrap_alphas, standard_error, ci_lower, ci_upper

# Performing bootstrapping with B = 1000
B = 1000
bootstrap_alphas, alpha_se, ci_lower, ci_upper = bootstrap_alpha(df, alpha_func, num_bootstrap=B)

print(f"Bootstrap alpha standard error: {alpha_se}")
print(f"95% confidence interval for alpha: [{ci_lower}, {ci_upper}]")


_(vi)_ What is the original mean value of `Sepal.Length`?

In [ ]:
# Your code here

Next, let's create a new bootstrapping to bootstrap samples (`boot_se_samples`) of `Sepal.Length`, in order to compute its bootstrapped mean and standard deviation.

_(vii)_. Write code to review the bootstrapped mean value, and the standard deviation of the bootstrapped samples. Compare the mean against its original value. Then, review the bootstrapped range, by using `t_range = np.ptp(boot_se_samples)`.

In [ ]:
# Add your code here

# Calculate the mean value of Sepal.Length
original_mean_sepal_length = df['Sepal.Length'].mean()
print(f"The original mean value of Sepal.Length is: {original_mean_sepal_length}")


In [ ]:
#bootstrapped mean value

# Setting the random seed for reproducibility
np.random.seed(0)

# Function to perform bootstrapping
def bootstrap_sepal_length(data, n_bootstrap_samples=1000):
    boot_se_samples = []
    n = len(data)
    for _ in range(n_bootstrap_samples):
        sample = np.random.choice(data, size=n, replace=True)
        boot_se_samples.append(sample.mean())
    return np.array(boot_se_samples)

#  bootstrapping
sepal_length = df['Sepal.Length'].values
boot_se_samples = bootstrap_sepal_length(sepal_length)

# Calculating bootstrapped mean and standard deviation
boot_mean = boot_se_samples.mean()
boot_std = boot_se_samples.std()

# Original mean value
original_mean_sepal_length = sepal_length.mean()

# Calculating the range of the bootstrapped samples
t_range = np.ptp(boot_se_samples)

# Print the results
print(f"Original mean value of Sepal.Length: {original_mean_sepal_length}")
print(f"Bootstrapped mean value of Sepal.Length: {boot_mean}")
print(f"Bootstrapped standard deviation of Sepal.Length: {boot_std}")
print(f"Range of the bootstrapped samples (t_range): {t_range}")

_(viii)_ Next, let's compute 95% confidence intervals, for the mean value of iris sepal length. (Hint: use the `np.percentile` function)

In [ ]:
# Add your code here

# Calculating the 95% confidence intervals
conf_interval_95 = np.percentile(boot_se_samples, [2.5, 97.5])

# Print the results
print(f"Original mean value of Sepal.Length: {original_mean_sepal_length}")
print(f"Bootstrapped mean value of Sepal.Length: {boot_mean}")
print(f"Bootstrapped standard deviation of Sepal.Length: {boot_std}")
print(f"Range of the bootstrapped samples (t_range): {t_range}")
print(f"95% confidence intervals for Sepal.Length: {conf_interval_95}")

_(ix)_. Use the plot function to create an histogram of the bootstrapped samples. What does this histogram show ?

In [ ]:
#Complete this

# Create a figure and axis
fig, ax = plt.subplots()

# Create the histogram
#Add your code here
ax.hist(boot_se_samples, bins=30, edgecolor='k', alpha=0.7)

# Add a title
#Add your code here
ax.set_title('Histogram of Bootstrapped Sample Means of Sepal.Length')

# Add a label to the x-axis
#Add your code here
ax.set_xlabel('Bootstrapped Sample Means of Sepal.Length')

# Add a label to the y-axis
#Add your code here
ax.set_ylabel('Frequency')

# Show the plot
plt.show()


_(x)_ Given your bootstrapped analysis, what do you recommend to shipping company? 

#### Write your answer here
1. Considering the standard deviation and the 95% confidence interval, it is important to account for variability. The company should prepare for some fluctuation around the mean size, ensuring there is a buffer space to accommodate this variability.

2. The confidence interval indicates the range within which the true mean Sepal.Length is likely to fall. This can provide the company with a more robust estimate than just using the sample mean alone. For example, if the 95% confidence interval is [5.7, 6.0], the company can use this range to ensure they are not underestimating or overestimating the space required.

3. The company should inspect the dataset for any significant outliers and consider them in their planning to avoid any surprises.

# Criteria

|Criteria            |Complete           |Incomplete          |
|--------------------|---------------|--------------|
|Bootstrapping|All steps are done correctly and the answers are correct.|At least one step is done incorrectly leading to a wrong answer.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Note:

If you like, you may collaborate with others in the cohort. If you choose to do so, please indicate with whom you have worked with in your pull request by tagging their GitHub username. Separate submissions are required.


### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/applying_statistical_concepts/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.
